###Ingest PitStops data from bronze to silver

0. Import configuration notebook and widgets

In [0]:
%run ../Includes/Configuration

In [0]:
dbutils.widgets.text('p_data_source', '', 'Data Source')
v_data_source = dbutils.widgets.get('p_data_source')

1. Imports

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp, lit

2. Create schema

In [0]:
pitstop_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), False),
        StructField("stop", IntegerType(), False),
        StructField("lap", IntegerType(), False),
        StructField("time", StringType(), False),
        StructField("duration", StringType(), True),
        StructField("milliseconds", IntegerType(), True)
    ]
)

3. Read pit_stops.json file

In [0]:
pitstops_raw_df = spark.read \
    .format("json") \
    .option("multiline", "true") \
    .schema(pitstop_schema) \
    .load(f"{bronze_container_path}/pit_stops.json")

4. Transform the circuit dataframe

In [0]:
pitstops_df = pitstops_raw_df \
        .select(
                col("raceId").alias("race_id"),
                col("driverId").alias("driver_id"),
                col("stop"),
                col("lap"),
                col("time"),
                col("duration"),
                col("milliseconds"),
                current_timestamp().alias('ingestion_date')
        ) \
        .withColumn("data_source", lit(v_data_source))

5. Write the dataframe to silver layer

In [0]:
pitstops_df.write.mode('overwrite').parquet(f"{silver_container_path}/pitstops")